Reference:  https://www.kaggle.com/code/kkanda/analyzing-uci-crime-and-communities-dataset

In [59]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objs as go
#from plotly.offline import plot
from plotly.offline import init_notebook_mode,iplot
init_notebook_mode(connected=True)
#import statsmodels.api as sm
from sklearn import linear_model
from sklearn.model_selection import train_test_split,KFold,cross_val_score
import re
import json
# from pandas.io.json import json_normalize

In [60]:
# load the processed data 
crimedata = pd.read_csv('../../Data/processed/communities_and_crime.csv')
crimedata.head()

,state,communityname,fold,householdsize,racepctblack,racePctWhite,racePctAsian,racePctHisp,agePct12t21,agePct12t29,...,PctBornSameState,PctSameHouse85,PctSameCity85,PctSameState85,LandArea,PopDens,PctUsePubTrans,LemasPctOfficDrugUn,population,ViolentCrimesPerPop
0,8,Lakewoodcity,1,0.33,0.02,0.90,0.12,0.17,0.34,0.47,...,0.42,0.50,0.51,0.64,0.12,0.26,0.20,0.32,0.19,0.20
1,53,Tukwilacity,1,0.16,0.12,0.74,0.45,0.07,0.26,0.59,...,0.50,0.34,0.60,0.52,0.02,0.12,0.45,0.00,0.00,0.67
2,24,Aberdeentown,1,0.42,0.49,0.56,0.17,0.04,0.39,0.47,...,0.49,0.54,0.67,0.56,0.01,0.21,0.02,0.00,0.00,0.43
3,34,Willingborotownship,1,0.77,1.00,0.08,0.12,0.10,0.51,0.50,...,0.30,0.73,0.64,0.65,0.02,0.39,0.28,0.00,0.04,0.12
4,42,Bethlehemtownship,1,0.55,0.02,0.95,0.09,0.05,0.38,0.38,...,0.72,0.64,0.61,0.53,0.04,0.09,0.02,0.00,0.01,0.03


In [61]:
crimedata.shape

(1994, 104)

In [62]:
for column in crimedata.columns:
    print(column)

state
communityname
fold
householdsize
racepctblack
racePctWhite
racePctAsian
racePctHisp
agePct12t21
agePct12t29
agePct16t24
agePct65up
numbUrban
pctUrban
medIncome
pctWWage
pctWFarmSelf
pctWInvInc
pctWSocSec
pctWPubAsst
pctWRetire
medFamInc
perCapInc
whitePerCap
blackPerCap
indianPerCap
AsianPerCap
OtherPerCap
HispPerCap
NumUnderPov
PctPopUnderPov
PctLess9thGrade
PctNotHSGrad
PctBSorMore
PctUnemployed
PctEmploy
PctEmplManu
PctEmplProfServ
PctOccupManu
PctOccupMgmtProf
MalePctDivorce
MalePctNevMarr
FemalePctDiv
TotalPctDiv
PersPerFam
PctFam2Par
PctKids2Par
PctYoungKids2Par
PctTeen2Par
PctWorkMomYoungKids
PctWorkMom
NumIlleg
PctIlleg
NumImmig
PctImmigRecent
PctImmigRec5
PctImmigRec8
PctImmigRec10
PctRecentImmig
PctRecImmig5
PctRecImmig8
PctRecImmig10
PctSpeakEnglOnly
PctNotSpeakEnglWell
PctLargHouseFam
PctLargHouseOccup
PersPerOccupHous
PersPerOwnOccHous
PersPerRentOccHous
PctPersOwnOccup
PctPersDenseHous
PctHousLess3BR
MedNumBR
HousVacant
PctHousOccup
PctHousOwnOcc
PctVacantBoarded
Pc

In [63]:
crimedata = crimedata.rename(columns={'PctUsePubTransLemasPctOfficDrugUn': 'PctUsePubTrans'})


In [69]:
import us
# Function to convert FIPS state code to state name
def fips_to_state(fips_code):
    try:
        return us.states.lookup(str(fips_code).zfill(2)).name
    except:
        return None

In [72]:
# Convert state codes to state names (you may need to map this if the data uses numerical codes)
crimedata['state_name'] = crimedata['state'].apply(fips_to_state)
crimedata['state_name'].head()

0        Colorado
1      Washington
2        Maryland
3      New Jersey
4    Pennsylvania
Name: state_name, dtype: object

In [73]:
print("unique values in state_name: ", crimedata['state_name'].nunique())

unique values in state_name:  45


In [74]:
print("list all unique values in state_name: ", crimedata['state_name'].unique())

list all unique values in state_name:  ['Colorado' 'Washington' 'Maryland' 'New Jersey' 'Pennsylvania'
 'California' 'Rhode Island' 'Kentucky' 'Missouri' 'New York'
 'Massachusetts' 'Wisconsin' 'Iowa' 'Indiana' 'Florida' 'Oregon' 'Ohio'
 'West Virginia' 'Connecticut' 'Alabama' 'Virginia' 'Oklahoma'
 'New Mexico' 'New Hampshire' 'Texas' 'Arizona' 'North Carolina' 'Utah'
 'South Dakota' 'Mississippi' 'Louisiana' 'South Carolina' 'Idaho'
 'Tennessee' 'Arkansas' 'Vermont' 'Georgia' 'Maine' 'North Dakota'
 'Minnesota' 'Kansas' 'Wyoming' 'Alaska' None 'Nevada' 'Delaware']


In [76]:
import plotly.express as px
state_data = crimedata.groupby('state_name')['ViolentCrimesPerPop'].mean().reset_index()

fig = px.choropleth(state_data, 
                    locations='state_name', 
                    locationmode="USA-states", 
                    color='ViolentCrimesPerPop',
                    color_continuous_scale='Reds',
                    scope="usa",
                    labels={'ViolentCrimesPerPop':'Violent Crimes per Population'},
                    title="Average Violent Crimes per Population by State")
fig.show()


ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

In [78]:
crimedata['ViolentCrimesPerPop']

0       0.20
1       0.67
2       0.43
3       0.12
4       0.03
        ... 
1989    0.09
1990    0.45
1991    0.23
1992    0.19
1993    0.48
Name: ViolentCrimesPerPop, Length: 1994, dtype: float64